In [ ]:
import numpy as np

from detector_response import DetectorResponse
import matplotlib.pyplot as plt
plt.rcParams.update({"text.usetex": True, "font.size":16, "font.family": "sans"})


In [ ]:
D = DetectorResponse()

electron = D.load_single_particle_data('/home/mart/auger/gits/SingleParticleInjection_v2/data/disk_flat_lgE_UUB_SSD_Electron.txt')
muon = D.load_single_particle_data('/home/mart/auger/gits/SingleParticleInjection_v2/data/disk_flat_lgE_UUB_SSD_Muon.txt')
photon = D.load_single_particle_data('/home/mart/auger/gits/SingleParticleInjection_v2/data/disk_flat_lgE_UUB_SSD_Photon.txt')

In [ ]:
muon2GeV = D.load_single_particle_data('/home/mart/auger/gits/SingleParticleInjection_v2/data/disk_2GeV_UUB_SSD_Muon.txt')

In [ ]:
from utils import plot_boot_binned

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
bins = 10

#Muon
df = muon.loc[(muon['E'] >= 1e9) & (muon['VEMpeak'] > 0.01)]
plot_boot_binned(df['cos_theta'], df['VEMpeak'], ax=ax1, bins=bins, stat='mean', 
                color='b', label='muons $E_{kin} >= 1$ GeV')
# df = muon1GeV.loc[muon1GeV['VEMpeak'] > 0.01]
# plot_boot_binned(df['cos_theta'], df['VEMpeak'], ax=ax1, bins=bins, stat='mean', 
#                 color='b', label='muons $E_{kin} = 1$ GeV')

df = muon.loc[(muon['E'] > 100e6) & (muon['MIPpeak'] > 0.1)]
plot_boot_binned(df['cos_theta'], df['MIPpeak'], ax=ax2, bins=bins, color='b', 
                label='muons $E_{kin} > 100$ MeV');


#Electron
df = electron.loc[(electron['E'] > 300e6) & (electron['E'] < 400e6) & (electron['VEMpeak'] > 0.01)]
plot_boot_binned(df['cos_theta'], df['VEMpeak'], ax=ax1, bins=bins, stat='mean', 
                color='r', label='electrons $300 < E_{kin}/\\mathrm{MeV} < 400$')
df = electron.loc[(electron['E'] > 2e6) & (electron['MIPpeak'] > 0.1)]
plot_boot_binned(df['cos_theta'], df['MIPpeak'], ax=ax2, bins=bins, color='r',
                label='electrons $E_{kin} > 2$ MeV');

#Photons
df = photon.loc[(photon['E'] > 300e6) & (photon['E'] < 400e6) & (photon['VEMpeak'] > 0.01)]
plot_boot_binned(df['cos_theta'], df['VEMpeak'], ax=ax1, bins=bins, stat='mean', 
                color='y', label='photons $300 < E_{kin}/\\mathrm{MeV} < 400$')

theta = np.linspace(0, np.pi/3)
D.SSD = False
ax1.plot(np.cos(theta), D.TL(theta), 'k--', label='average track length WCD', lw=2)
D.SSD = True
ax2.plot(np.cos(theta), D.TL(theta), 'k:', label='average track length SSD', lw=2)  

for ax in [ax1, ax2]:
    ax.set_xlabel('$\\cos{\\chi}$')
#     ax.set_ylim([0.5, 2])
ax1.set_ylabel('VEM peak')
ax2.set_ylabel('MIP peak')
ax1.set_title('WCD')
ax2.set_title('SSD (hit)')
ax1.legend()
ax2.legend()
plt.savefig('detector_TL_response.png', dpi=300, bbox_inches='tight')

In [ ]:
f, ax = plt.subplots(1)
bins = 10

#Muon
df = muon.loc[(muon['E'] < 400e6) & (muon['VEMpeak'] > 0)]
plot_boot_binned(df['cos_theta'], df['VEMpeak'], ax=ax, bins=bins, stat='mean', 
                color='b', label='muons $E_{kin} < 400$ MeV');

ax.set_xlabel('$\\cos{\\chi}$')
ax.set_ylabel('VEM peak')

ax.legend()
plt.savefig('detector_TL_response_muon<400MeV.png', dpi=300, bbox_inches='tight')

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))

colors = ['b', 'r', 'y']
labels = ['muons', 'electrons', 'photons']

for i, df in enumerate([muon, electron, photon]):

    ax1.scatter(df['E'], df['VEMpeak'], marker='.', alpha=0.2, color=colors[i])
    ax2.scatter(df['E'], df['MIPpeak'], marker='.', alpha=0.2, color=colors[i])
    
    
Espace = np.logspace(np.log10(2e6), np.log10(2e9))

def curve_EM(E, Evem):
    return E/Evem

def curve_muon(E, Evem):
    return np.heaviside(E-Evem, 1) + np.heaviside(Evem-E, 1)*(E/Evem)**2


Evem = 380e6
ax1.plot(Espace, curve_EM(Espace, Evem), 'r-', lw=4)
ax1.plot(Espace, curve_muon(Espace, Evem), 'b-', lw=4)

ax2.plot(Espace, np.ones_like(Espace), 'b-', lw=4)
ax2.plot(Espace, np.ones_like(Espace), 'r--', lw=3)

from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='b', marker='o', ls=''),
                Line2D([0], [0], color='r', marker='o', ls=''),
                Line2D([0], [0], color='y', marker='o', ls=''),
                Line2D([0], [0], color='b', ls='-'),
                Line2D([0], [0], color='r', ls='-')]

ax1.legend(custom_lines, ['muons', 'electrons', 'photons', 'muon model', 'EM model'])

for ax in [ax1, ax2]:
    ax.set_ylim([1e-2, 10])

    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('$E_{\\mathrm{kin}}$ [eV]')
    
ax1.set_ylabel('VEM peak')
ax2.set_ylabel('MIP peak')
ax1.set_title('WCD')
ax2.set_title('SSD')
plt.savefig('detector_E_response.png', bbox_inches='tight', dpi=300)